In [ ]:
import requests
import pandas as pd
import datetime as dt
import xml.etree.ElementTree as ET
import re

# =====================================================================
# 1. USER SETTINGS (THE ONLY PART YOU EVER EDIT)
# =====================================================================

# ---- Nomad IPTV Paid Credentials (FULL VERSION ONLY) ----
paid_username = "Nact6578"
paid_password = "Earm3432"

paid_url = f"http://nomadiptv.online:25461/get.php?username={paid_username}&password={paid_password}&type=m3u_plus&output=ts"

# ---- Free Playlists (BOTH VERSIONS USE THESE) ----
free_playlists = [
    "https://raw.githubusercontent.com/BuddyChewChew/ppv/refs/heads/main/PPVLand.m3u8",
    "https://raw.githubusercontent.com/BuddyChewChew/My-Streams/refs/heads/main/Pixelsports.m3u8",
    "https://raw.githubusercontent.com/BuddyChewChew/My-Streams/refs/heads/main/StreamSU.m3u",
    "https://raw.githubusercontent.com/BuddyChewChew/My-Streams/refs/heads/main/Backup.m3u",
    "https://raw.githubusercontent.com/BuddyChewChew/buddylive-combined/refs/heads/main/combined_playlist.m3u",
    "https://raw.githubusercontent.com/BuddyChewChew/buddylive/refs/heads/main/en/videoall.m3u",
    "https://raw.githubusercontent.com/BuddyChewChew/iptv/refs/heads/main/M3U8/events.m3u8",
]

# ---- EPG (Optional but recommended for “live now” accuracy) ----
epg_urls = [
    # "https://raw.githubusercontent.com/Addicted2u143/Mega-EPG/main/combined_epg_latest.xml.gz",
]

# =====================================================================
# 2. CATEGORY DEFINITIONS (YOU APPROVED THESE)
# =====================================================================

SPORT_KEYWORDS = {
    "🏈 Football (NFL + NCAA FB)": [
        "nfl", "football", "redzone", "ncaaf", "acc network", "sec network",
        "big ten", "b1g", "college football"
    ],
    "🏀 Basketball (NBA + NCAA BB)": [
        "nba", "basketball", "ncaab", "college basketball", "march madness"
    ],
    "⚾ Baseball (MLB)": ["mlb", "baseball"],
    "🏒 Hockey (NHL)": ["nhl", "hockey"],
    "⚽ Soccer": ["soccer", "futbol", "premier", "laliga", "ucl", "mls", "bundesliga"],
    "🥊 Combat Sports (UFC/Boxing/WWE/PPV)": ["ufc", "boxing", "wwe", "mma", "ppv", "fight"],
    "🏎 Motorsports": ["nascar", "f1", "formula 1", "indycar", "motogp"],
    "🎾 Golf / Tennis": ["golf", "tennis", "pga", "atp", "wta"],
    "📺 General Sports": ["espn", "fox sports", "tsn", "bein", "sportsnet", "sky sports"],
}

LIVE_HINTS = ["live", "live now", "in progress", "on air", "pregame", "postgame"]

EVENT_HINTS = ["event", "fight", "ufc", "ppv", "round", "match"]


# =====================================================================
# 3. CORE HELPERS
# =====================================================================

def download_m3u(url):
    try:
        print(f"Fetching: {url}")
        return requests.get(url, timeout=20).text
    except:
        print("Failed:", url)
        return ""

def parse_m3u(text):
    lines, out = text.splitlines(), []
    name = logo = group = tvg_id = None
    for line in lines:
        if line.startswith("#EXTINF"):
            logo = group = tvg_id = ""
            if 'tvg-logo="' in line:
                logo = line.split('tvg-logo="')[1].split('"')[0]
            if 'group-title="' in line:
                group = line.split('group-title="')[1].split('"')[0]
            if 'tvg-id="' in line:
                tvg_id = line.split('tvg-id="')[1].split('"')[0]
            name = line.split(",")[-1].strip()
        elif line.startswith("http"):
            out.append([name, line.strip(), logo, group, tvg_id])
    return pd.DataFrame(out, columns=["name", "url", "logo", "group", "tvg_id"])


def classify_category(name, group):
    combo = f"{name} {group}".lower()
    for cat, keys in SPORT_KEYWORDS.items():
        if any(k in combo for k in keys):
            return cat
    return None


def looks_live(name, group):
    combo = f"{name} {group}".lower()
    return any(k in combo for k in LIVE_HINTS + EVENT_HINTS)


# =====================================================================
# 4. COMBINE PLAYLISTS (FULL + FREE)
# =====================================================================

def load_full_playlist():
    dfs = []
    paid = download_m3u(paid_url)
    if paid.strip():
        dfs.append(parse_m3u(paid))
    for url in free_playlists:
        t = download_m3u(url)
        if t.strip():
            dfs.append(parse_m3u(t))
    return pd.concat(dfs, ignore_index=True)

def load_free_playlist():
    dfs = []
    for url in free_playlists:
        t = download_m3u(url)
        if t.strip():
            dfs.append(parse_m3u(t))
    return pd.concat(dfs, ignore_index=True)


# =====================================================================
# 5. CATEGORY + LIVE NOW LOGIC
# =====================================================================

def categorize(df):
    df = df.copy()
    df["category"] = df.apply(lambda r: classify_category(r["name"], r["group"]), axis=1)
    df = df[df["category"].notna()]  # keep only sports
    df["is_live_hint"] = df.apply(lambda r: looks_live(r["name"], r["group"]), axis=1)
    return df


def build_sorted(df):
    live_now = df[df["is_live_hint"]].copy()
    non_live = df[~df["is_live_hint"]].copy()

    # Sort live first by name
    live_now = live_now.sort_values("name")

    # Sort inside categories: live first then name
    out = []
    out.append(("🔥 LIVE NOW", live_now))

    for cat in SPORT_KEYWORDS.keys():
        block = df[df["category"] == cat]
        if block.empty:
            continue
        block_live = block[block["is_live_hint"]].sort_values("name")
        block_rest = block[~block["is_live_hint"]].sort_values("name")
        out.append((cat, pd.concat([block_live, block_rest])))
    return out


# =====================================================================
# 6. EXPORT
# =====================================================================

def export_m3u(blocks, path):
    with open(path, "w") as f:
        f.write("#EXTM3U\n")
        for cat, df in blocks:
            f.write(f"#EXTINF:-1 group-title=\"{cat}\",{cat}\n")
            f.write("http://example.com/blank\n")
            for _, r in df.iterrows():
                f.write(
                    f'#EXTINF:-1 tvg-id="{r.tvg_id}" tvg-logo="{r.logo}" '
                    f'group-title="{cat}",{r.name}\n'
                )
                f.write(r.url + "\n")


# =====================================================================
# 7. RUN FULL + FREE
# =====================================================================

print("\nLoading full playlist...")
full_df = categorize(load_full_playlist())

print("Loading free-only playlist...")
free_df = categorize(load_free_playlist())

print("Building sorted playlists...")
full_blocks = build_sorted(full_df)
free_blocks = build_sorted(free_df)

print("Exporting...")
export_m3u(full_blocks, "/content/sports_master.m3u")
export_m3u(free_blocks, "/content/sports_master_free.m3u")

print("\nDone! Files created:")
print("/content/sports_master.m3u")
print("/content/sports_master_free.m3u")


Loading full playlist...
Fetching: http://nomadiptv.online:25461/get.php?username=Nact6578&password=Earm3432&type=m3u_plus&output=ts
Fetching: https://raw.githubusercontent.com/BuddyChewChew/ppv/refs/heads/main/PPVLand.m3u8
Fetching: https://raw.githubusercontent.com/BuddyChewChew/My-Streams/refs/heads/main/Pixelsports.m3u8
Fetching: https://raw.githubusercontent.com/BuddyChewChew/My-Streams/refs/heads/main/StreamSU.m3u
Fetching: https://raw.githubusercontent.com/BuddyChewChew/My-Streams/refs/heads/main/Backup.m3u
Fetching: https://raw.githubusercontent.com/BuddyChewChew/buddylive-combined/refs/heads/main/combined_playlist.m3u
Fetching: https://raw.githubusercontent.com/BuddyChewChew/buddylive/refs/heads/main/en/videoall.m3u
Fetching: https://raw.githubusercontent.com/BuddyChewChew/iptv/refs/heads/main/M3U8/events.m3u8
Loading free-only playlist...
Fetching: https://raw.githubusercontent.com/BuddyChewChew/ppv/refs/heads/main/PPVLand.m3u8
Fetching: https://raw.githubusercontent.com/Bud